In [3]:

# This cell is to load user and password of my database from an .txt file, which will not be pushed to the repository
# Assign your user name and password to lldb_user and lldb_password variables
with open('db_credentials.txt','r') as file:
    lines = file.readlines()
lldb_user = lines[0]
lldb_password = lines[1]
    

In [4]:
import mysql.connector as connector
from mysql.connector import Error

In [5]:
try:
    connection = connector.connect(user = lldb_user, password = lldb_password, db = 'littlelemondb')
except Error as e:
    print('An error has occurred with an error code: {} \n and message: {}'.format(e.errno,e.msg))

In [6]:
cursor = connection.cursor()
cursor.execute('SHOW tables')

In [7]:
results = cursor.fetchall()
for table in results:
    print(table)

('bookings',)
('customers',)
('deliveries',)
('items',)
('menuitems',)
('menus',)
('orders',)
('ordersview',)
('staff',)


In [8]:
cursor.execute("""
SELECT CONCAT(bc.FirstName, ' ', bc.LastName) AS 'Customer Full Name', bc.Email, bc.Phone, o.TotalCost
FROM Orders AS o 
LEFT JOIN 
	(SELECT b.BookingID, c.CustomerID, c.FirstName, c.LastName, c.Email, c.Phone
    FROM Bookings AS b
    LEFT JOIN Customers AS c 
    ON b.CustomerID = c.CustomerID) AS bc
ON o.BookingID = bc.BookingID
WHERE o.TotalCost > 60;
""")

print(cursor.column_names)

for record in cursor.fetchall():
    print (record)

('Customer Full Name', 'Email', 'Phone', 'TotalCost')
('Joakim Iversen', 'j.iversen@yahoo.com', 352253781, Decimal('92'))
('Diana Pinto', 'diana.pinto@finance.org.us', 751100242, Decimal('78'))
('Diana Pinto', 'diana.pinto@finance.org.us', 751100242, Decimal('120'))
('Marcos Romero', 'marcos.romero2208@mangatagallo.us', 451252112, Decimal('184'))


In [10]:
# Close the connection
if connection.is_connected():
    cursor.close()
    connection.close()
    print("Connection has been closed")
else:
    print("Connection was closed")

Connection was closed
